In [1]:
import tensorflow as tf
import cv2
import numpy as np
import os
from pathlib import Path

2025-10-18 14:21:53.486982: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-18 14:21:53.520626: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-18 14:21:54.720690: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
def find_project_root(markers=("pyproject.toml", ".git")) -> Path:
    """
    Descobre a raiz do projeto subindo diretórios até encontrar um marcador.

    Parâmetros:
        markers: nomes de arquivos/pastas que indicam a raiz (ex.: "pyproject.toml", ".git").

    Retorna:
        pathlib.Path apontando para a raiz do projeto, ou o diretório atual como fallback.
    """
    p = Path.cwd().resolve()
    for parent in [p, *p.parents]:
        if any((parent / m).exists() for m in markers):
            return parent
    return p

ROOT = find_project_root()

In [3]:
# --- Configurações ---
DATA_DIR = ROOT / "data"
MODEL_PATH = DATA_DIR / "processed" / "modelo_ocr_simbolos.keras"
IMAGE_PATH = DATA_DIR / "raw" / "ocr_test_image.png"
CLASS_NAMES_PATH = DATA_DIR / "raw" / "class_names.json"


IMG_HEIGHT = 64
IMG_WIDTH = 64

In [4]:
# Mapeamento de índice para classe. Deve corresponder à ordem do treinamento.
import json

with open(CLASS_NAMES_PATH, 'r', encoding='utf-8') as f:
    loaded_classes = json.load(f)

CLASS_SYMBOLS = loaded_classes


In [5]:
def preprocess_for_prediction(image_crop):
    """Prepara um recorte de imagem para o modelo."""
    # Redimensiona para o tamanho esperado pelo modelo
    resized_img = cv2.resize(image_crop, (IMG_WIDTH, IMG_HEIGHT))

    # Adiciona a dimensão do batch e do canal (o modelo espera 4D: batch, H, W, canal)
    img_array = np.expand_dims(resized_img, axis=-1) # Adiciona canal
    img_array = np.expand_dims(img_array, axis=0)    # Adiciona batch

    # Normaliza os pixels (se o modelo foi treinado com Rescaling, não precisa dividir por 255.0 aqui)
    # A camada Rescaling(1./255) no modelo já faz isso.
    return img_array

In [6]:
def show_image(cv2, image, title="Image"):
    """Exibe uma imagem usando OpenCV."""
    cv2.imshow('Imagem de Entrada', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [7]:
if not os.path.exists(MODEL_PATH):
    print(f"ERRO: Modelo '{MODEL_PATH}' não encontrado. Treine o modelo primeiro.")
    # return
if not os.path.exists(IMAGE_PATH):
    print(f"ERRO: Imagem de teste '{IMAGE_PATH}' não encontrada. Execute 'gerar_imagem_teste.py' primeiro.")
    # return

In [8]:
# 1. Carregar o modelo treinado
print("Carregando modelo...")
model = tf.keras.models.load_model(MODEL_PATH)


Carregando modelo...


I0000 00:00:1760808115.454055   57367 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5561 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


In [9]:

# 2. Carregar e pré-processar a imagem de entrada
image = cv2.imread(IMAGE_PATH)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)


In [12]:

# Binarização da imagem (inversa, para ter símbolos brancos em fundo preto)
_, thresh = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY_INV)

# (Opcional) Dilatar para conectar partes quebradas dos símbolos
kernel = np.ones((2,2), np.uint8)
thresh = cv2.dilate(thresh, kernel, iterations=1)

# Exibir a imagem binarizada (opcional)
show_image(cv2, thresh, title="Imagem Binarizada")

In [ ]:

# 3. Encontrar contornos
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)


bounding_boxes = []
for contour in contours:
    # Pega o retângulo envolvente de cada contorno
    x, y, w, h = cv2.boundingRect(contour)

    # Filtra contornos muito pequenos (ruído)
    if w > 10 and h > 10:
        bounding_boxes.append((x, y, w, h))

# Exibe os contornos encontrados (opcional)
# debug_image = image.copy()
# for (x, y, w, h) in bounding_boxes:
#     cv2.rectangle(debug_image, (x, y), (x + w, y + h), (0, 255, 0), 2)
# show_image(cv2, debug_image, title="Contornos Detectados")

error: OpenCV(4.12.0) :-1: error: (-5:Bad argument) in function 'findContours'
> Overload resolution failed:
>  - findContours() missing required argument 'mode' (pos 2)
>  - findContours() missing required argument 'mode' (pos 2)


In [21]:
# 4. Ordenar os retângulos da esquerda para a direita
bounding_boxes.sort(key=lambda b: b[0])

recognized_sequence = ""


# 5. Iterar, recortar e prever
for x, y, w, h in bounding_boxes:
    # Recorta o símbolo da imagem em escala de cinza original
    # Adiciona um padding para garantir que o símbolo inteiro seja capturado
    padding = 0
    symbol_crop = gray[y - padding : y + h + padding, x - padding : x + w + padding]

    # Exibe a imagem recortada (opcional)
    show_image(cv2, symbol_crop, title="Símbolo Recortado")


    # Prepara a imagem para o modelo
    # processed_crop = preprocess_for_prediction(symbol_crop)

    resized_img = cv2.resize(symbol_crop, (IMG_WIDTH, IMG_HEIGHT))

    # exibe a imagem recortada (opcional)
    # show_image(cv2, resized_img, title="Símbolo Recortado")

    # Adiciona a dimensão do batch e do canal (o modelo espera 4D: batch, H, W, canal)
    img_array = np.expand_dims(resized_img, axis=-1)  # Adiciona canal
    img_array = np.expand_dims(img_array, axis=0)  # Adiciona batch

    # Realiza a predição
    prediction = model.predict(img_array, verbose=0)

    # Obtém o símbolo com a maior probabilidade
    predicted_index = np.argmax(prediction)
    predicted_symbol = CLASS_SYMBOLS[predicted_index]

    recognized_sequence += predicted_symbol + ' '

    # Desenha o retângulo e o resultado na imagem original para visualização
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    # cv2.putText(image, predicted_symbol, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.2, (0, 255, 0), 2)

print(f"\nSequência Reconhecida: {recognized_sequence}")


# Palavra "Rosa" em ELiS
# U+0074, U+0068, U+007A, U+0044, U+00CC, U+00B0 em Unicode
# "thzDÌm" em ASCII


Sequência Reconhecida: U+00A2 U+007A U+0034 U+0034 U+00CC 


In [22]:

# 6. Exibir o resultado visual
cv2.imshow('Reconhecimento de Sequencia', image)
cv2.waitKey(0) # Espera uma tecla ser pressionada
cv2.destroyAllWindows()